# Classification: KNeighbors and Logistic regression

In [1]:
import sys
sys.path.append('..')

In [2]:
from source.code.data_loader import DataLoader
import pandas as pd

## Initial data loading

In [3]:
with open('../data/description/datasets', 'r') as source:
    datasets = source.readlines()
datasets = [dataset.replace('\n', '') for dataset in datasets]
dataset_path_pattern = '../data/{}/{}.{}.txt'
data_paths = [dataset_path_pattern.format('train', dataset, 'data') for dataset in datasets] + [dataset_path_pattern.format('test', dataset, 'test') for dataset in datasets]
data_loader = DataLoader(data_paths, '../data/description/columns', '../data/description/classes')
data_frame = data_loader.load()

## EDA

In [4]:
data_frame.head(30)

,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,...,TT4 measured,TT4,T4U measured,T4U,FTI measured,FTI,TBG measured,TBG,referral source,diagnosis
0,41.0,F,False,False,False,False,False,False,False,False,...,True,125.0,True,1.14,True,109.0,False,None,SVHC,0
1,23.0,F,False,False,False,False,False,False,False,False,...,True,102.0,False,NaN,False,NaN,False,None,other,0
2,46.0,M,False,False,False,False,False,False,False,False,...,True,109.0,True,0.91,True,120.0,False,None,other,0
3,70.0,F,True,False,False,False,False,False,False,False,...,True,175.0,False,NaN,False,NaN,False,None,other,0
4,70.0,F,False,False,False,False,False,False,False,False,...,True,61.0,True,0.87,True,70.0,False,None,SVI,0
5,18.0,F,True,False,False,False,False,False,False,False,...,True,183.0,True,1.30,True,141.0,False,None,other,0
6,59.0,F,False,False,False,False,False,False,False,False,...,True,72.0,True,0.92,True,78.0,False,None,other,0
7,80.0,F,False,False,False,False,False,False,False,False,...,True,80.0,True,0.70,True,115.0,False,None,SVI,15
8,66.0,F,False,False,False,False,False,False,False,False,...,True,123.0,True,0.93,True,132.0,False,None,SVI,0
9,68.0,M,False,False,False,False,False,False,False,False,...,True,83.0,True,0.89,True,93.0,False,None,SVI,0


In [7]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22632 entries, 0 to 971
Data columns (total 30 columns):
age                          22626 non-null float64
sex                          21732 non-null object
on thyroxine                 22632 non-null bool
query on thyroxine           22632 non-null bool
on antithyroid medication    22632 non-null bool
sick                         22632 non-null bool
pregnant                     22632 non-null bool
thyroid surgery              22632 non-null bool
I131 treatment               22632 non-null bool
query hypothyroid            22632 non-null bool
query hyperthyroid           22632 non-null bool
lithium                      22632 non-null bool
goitre                       22632 non-null bool
tumor                        22632 non-null bool
hypopituitary                22632 non-null bool
psych                        22632 non-null bool
TSH measured                 22632 non-null bool
TSH                          20418 non-null float64
T3 

После объединения могло возникнуть некоторое количество дупликатов, попробуем от них избавиться:

In [10]:
data_frame.drop_duplicates(inplace=True)

In [11]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4680 entries, 0 to 909
Data columns (total 30 columns):
age                          4679 non-null float64
sex                          4495 non-null object
on thyroxine                 4680 non-null bool
query on thyroxine           4680 non-null bool
on antithyroid medication    4680 non-null bool
sick                         4680 non-null bool
pregnant                     4680 non-null bool
thyroid surgery              4680 non-null bool
I131 treatment               4680 non-null bool
query hypothyroid            4680 non-null bool
query hyperthyroid           4680 non-null bool
lithium                      4680 non-null bool
goitre                       4680 non-null bool
tumor                        4680 non-null bool
hypopituitary                4680 non-null bool
psych                        4680 non-null bool
TSH measured                 4680 non-null bool
TSH                          4347 non-null float64
T3 measured           

Судя по всему подавляющее большинство информации в датастах было дуплицировано.

Из описания видно, что признак TBG не содержит ни одного ненулевого значения, соответственно значение данного признакане было измерено ни для одного пациента. Данный признак (а соответственно и признак TBG measured) можно удалить из выборки:

In [13]:
data_frame.drop(['TBG measured', 'TBG'], axis='columns', inplace=True)

Referal source также вряд ли привнесет много полезного, поскольку является не столько информацией о пациентах, сколько информацией об информации о пациентах, так что пока удалим этот признак из выборки:

In [14]:
data_frame.drop(['referral source'], axis='columns', inplace=True)

In [15]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4680 entries, 0 to 909
Data columns (total 27 columns):
age                          4679 non-null float64
sex                          4495 non-null object
on thyroxine                 4680 non-null bool
query on thyroxine           4680 non-null bool
on antithyroid medication    4680 non-null bool
sick                         4680 non-null bool
pregnant                     4680 non-null bool
thyroid surgery              4680 non-null bool
I131 treatment               4680 non-null bool
query hypothyroid            4680 non-null bool
query hyperthyroid           4680 non-null bool
lithium                      4680 non-null bool
goitre                       4680 non-null bool
tumor                        4680 non-null bool
hypopituitary                4680 non-null bool
psych                        4680 non-null bool
TSH measured                 4680 non-null bool
TSH                          4347 non-null float64
T3 measured           

Заметим, что TSH measured, T3 measured, TT4 measured, T4U measured и FTI measured просто показывают, было ли измерено у пациента значение признаков TSH, T3, TT4, T4U и FTI соответственно. Данная информация избыточна, поскольку по значению этих признаков можно понять были они измерены или нет (наличие или отсутствие NaN), TSH measured, T3 measured, TT4 measured, T4U measured и FTI measured также можно удалить из выборки:

In [16]:
data_frame.drop(['TSH measured', 'T3 measured', 'TT4 measured', 'T4U measured', 'FTI measured'], axis='columns', inplace=True)

In [17]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4680 entries, 0 to 909
Data columns (total 22 columns):
age                          4679 non-null float64
sex                          4495 non-null object
on thyroxine                 4680 non-null bool
query on thyroxine           4680 non-null bool
on antithyroid medication    4680 non-null bool
sick                         4680 non-null bool
pregnant                     4680 non-null bool
thyroid surgery              4680 non-null bool
I131 treatment               4680 non-null bool
query hypothyroid            4680 non-null bool
query hyperthyroid           4680 non-null bool
lithium                      4680 non-null bool
goitre                       4680 non-null bool
tumor                        4680 non-null bool
hypopituitary                4680 non-null bool
psych                        4680 non-null bool
TSH                          4347 non-null float64
T3                           3839 non-null float64
TT4                

Заметим, что в данных присутствуют записи о неопределившихся с полом:

In [31]:
data_frame[(data_frame.sex.isnull())].sort_index().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185 entries, 16 to 2786
Data columns (total 22 columns):
age                          185 non-null float64
sex                          0 non-null object
on thyroxine                 185 non-null bool
query on thyroxine           185 non-null bool
on antithyroid medication    185 non-null bool
sick                         185 non-null bool
pregnant                     185 non-null bool
thyroid surgery              185 non-null bool
I131 treatment               185 non-null bool
query hypothyroid            185 non-null bool
query hyperthyroid           185 non-null bool
lithium                      185 non-null bool
goitre                       185 non-null bool
tumor                        185 non-null bool
hypopituitary                185 non-null bool
psych                        185 non-null bool
TSH                          166 non-null float64
T3                           157 non-null float64
TT4                          170 non-n

Глянем, есть ли среди неопределившихся с полом беременные:

In [32]:
data_frame[(data_frame.sex.isnull()) & (data_frame.pregnant)]

,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,...,goitre,tumor,hypopituitary,psych,TSH,T3,TT4,T4U,FTI,diagnosis
1609,73.0,None,False,False,False,False,True,False,False,False,...,False,False,False,False,2.2,2.5,110.0,1.28,85.0,0
471,21.0,None,False,False,False,False,True,False,False,False,...,False,True,False,False,2.4,3.5,171.0,1.49,115.0,0
471,21.0,None,False,False,False,False,True,False,False,False,...,False,True,False,False,2.4,3.5,171.0,1.49,115.0,4


Поскольку пол - довольно важный признак и поскольку неопределившиеся составляют $100 \cdot \frac{185}{4680} \approx 4\%$, будет проще избавиться от данной части выборки:

In [ ]:
data_frame[(data_frame.sex.isnull()) & (data_frame.pregnant)].index